# Hepsiburada Duygu Analizi 

hepsiburada.com üzerinden yaklaşık 250.000 yorum derlenmiş ve yorumların sahip olduğu yıldız sayısına göre pozitif veya negatif olarak etiketlenmiştir.

*   1 veya 2 yıldız : olumsuz yorum (0)
*   4 veya 5 yıldız : olumlu yorum  (1)
*   3 yıldıza sahip yorumlar veri kümesinden çıkarılmıştır.

**Böylelikle toplamda analiz edilecek yorum sayısı : 243.496**

**Yorumların %90'dan fazlası olumlu yoruma sahiptir.**






In [ ]:
 import numpy as np
import pandas as pd

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
DATA_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day14-NLP/notebooks/datasets/"

DATASET_PATH = DATA_DIR + "hepsiburada.csv"

In [ ]:
dataset = pd.read_csv(DATASET_PATH)

In [ ]:
dataset

**Etiket değerleri ve analiz edilecek yorumları liste olarak alalım.**

*   Hedef değişkeni : 'Rating'
*   Analiz edilecek yorumlar : 'Review' 


In [ ]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

**Eğitim ve Test kümelerini ayıralım.**

Elimizdeki verilerin %80'inini eğitim, %20'sini ise test için ayıralım.

In [ ]:
cutoff = int(len(data) * 0.80)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [ ]:
x_train[500]

In [ ]:
x_train[800]

In [ ]:
y_train[800]

# Tokenleştirme (Tokenization) 

Her yorumu kelimelere ayrılacak ve kelime haznesindeki her bir kelimeye bir sayı denk gelecektir.

Öncelikle kelime haznemizde en fazla bulunacak kelime sayısını **(num_words=10.000)** tanımlayalım. Böylelikle veri kümesi içerisinde en sık geçen 10.000 kelimeyi ele alacağız. Bu kelimelerin dışında kalan nadir kelimeleri ise yok sayacağız.

Not : Herhangi bir sınır belirlemek istemezseniz 'None' yazabilir veya default olarak bırakabilirsiniz.

In [ ]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
tokenizer.fit_on_texts(data)

# Kelime haznemizi görüntüleyelim.

In [ ]:
tokenizer.word_index

**Eğitim ve Test kümesindeki tüm yorumları tokenler halinde bir değişkende saklayalım.**


In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)

Eğitim kümesindeki 800. yorumu ele alalım. Yorumu görüntüleyip, tokenleştirme sonrası çıktısına bakalım. Yukarıdaki listeden de kontrol ettiğimizde örneğin 'ürünü' kelimesinin 19'a karşılık geldiğini görebiliriz.

Kelime haznesini 10.000 ile sınırladığımız için, bu sınırın dışında kalan nadir kelimeler tokenleştirilemeyecektir. Örneğin 800. yorum içerisinde geçen **'bilgisyarım'** kelimesi yazım yanlışı barındıran nadir bir kelimedir, bu sebeple en sık geçen 10.000 kelime içerisinde yoktur. Bu sebeple tokenleştirilememiştir ve bu kelime yok sayılmıştır.

In [ ]:
x_train[800]

In [ ]:
print(x_train_tokens[800])

In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

Her yorumda bulunan token sayısını bulalım.


In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

Yorumlarda ortalama 20 token bulunmaktadır.

In [ ]:
np.mean(num_tokens)

En uzun yorumun token sayısını bulalım.

In [ ]:
np.max(num_tokens)

In [ ]:
np.argmax(num_tokens)

In [ ]:
x_train[21941]

# Padding boyutunu belirleyelim.
Bunun birçok yolu var. İsteğe bağlı olarak maksimum token sayısına veya ortalama token sayısını eşitlenebilir. Ancak burada, biraz daha matematiksel bir tercih yapılacaktır.

**max_tokens = num_tokens'ın ortalaması + 2* (num_tokens'ın standart sapması)**

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

Bu hesap sonucunda çıkan 59 token, yorumların yüzde kaçını kapsamaktadır?

In [ ]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

In [ ]:
len(x_train_tokens)

# Padding İşlemi 

RNN ile oluşturduğumuz modellere belli boyutlarda input'lar veririz.
Üzerinde çalıştığımız datasetteki yorumların ise boyutları birbirinden farklıdır. Bu sebeple padding işlemi ile tüm yorumların boyutları eşitlenecektir.

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)

In [ ]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [ ]:
x_train_pad.shape

In [ ]:
x_test_pad.shape

In [ ]:
np.array(x_train_tokens[800])

In [ ]:
x_train_pad[800]

## Token'leri verip string'e çevirelim. 

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [ ]:
x_train[800]

In [ ]:
tokens_to_string(x_train_tokens[800])

## YSA Modelimizi Oluşturalım

In [ ]:
model = Sequential()

Her kelimeye karşılık gelen vektör uzunluğunu belirleyelim.
Kelime vektörleri başlangıçta rastgele oluşturacağız, ancak model değitilirken vektörlerde eğitiliyor olacak.

In [ ]:
embedding_size = 50

In [ ]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [ ]:
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimizer = Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
x_train_pad = np.array(x_train_pad)
y_train = np.array(y_train)

x_test_pad = np.array(x_test_pad)
y_test = np.array(y_test)

In [ ]:
model.fit(x_train_pad, y_train, epochs=5, batch_size=256)

In [ ]:
result = model.evaluate(x_test_pad, y_test)

In [ ]:
result[1]

In [ ]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

In [ ]:
cls_true = np.array(y_test[0:1000])

In [ ]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [ ]:
len(incorrect)

In [ ]:
idx = incorrect[0]
idx

In [ ]:
text = x_test[idx]
text

In [ ]:
y_pred[idx]

In [ ]:
cls_true[idx]

In [ ]:
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [ ]:
tokens = tokenizer.texts_to_sequences(texts)

In [ ]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

In [ ]:
model.predict(tokens_pad)